<h1>Solving the Traveling Santa Problem with an Ants Colony Algorithm</h1>

An Ants Colony Algorithm is given to solve the challenge set in Kaggle described [here] (https://www.kaggle.com/c/traveling-santa-problem) and quoted below:


>Santa asked that we give you one particular instance of his TSP (Traveling Santa Problem). However, Santa's dilemma isn't quite the same as the Traveling Salesman Problem with which you may be familiar. Santa likes to see new terrain every year--don't ask, it's a reindeer thing--and doesn't want his route to be predictable.

>You're looking for shortest-distance paths through a set of chimneys, but instead of providing one path, Santa asks you to provide two disjoint paths. If one of your paths contains an edge from A to B, your other path must not contain an edge from A to B or from B to A (either order still counts as using that edge). Your score is the larger of the two distances. 

The general approach is using two different competing colonies, each to find one of the required cheminies. As the challenge requires to minimize the lenght of the longest cheminy, we do not really want the two colonies to compete. Instead, we want one of them to find the best path ever, and the second one to find the second best path. 

The challenge provides as source data a CSV file with 150.000 elements with the following very simple structure:
- name of a city (node)
- latitude 
- longitude 

<h2>General description of the algoithm</h2>

The source data set has been imported and represented as a **weighted, oriented graph**, where *each node represents a city*, and *each edge represents an atomic step*, whose weight is the euclidean distance between the two connected cities. 

The algorithm is based on two phases:
1) Exploration of the graph space (**exploring ants**)
2) Hunting the solution (two cheminies) to the challenge (**hunting ants**)

During the first phase ants visit the graph by chosing at each node a random edge between those leading to the *n* closest, not yet visited nodes. *n* is a parameter of the algorithm. The main purpose of this solution is leaving feromone on edges.

During the second phase, ants prioritize edges based on the amount of feromone. When the feromone left on a certain edge by a certain colony is strong enough, it is said to be a **prime edge**. A prime edge is assigned to the final solution for a certain colony. The final solution is sequentially composed ant by ant, iteration after iteration, and it is not identified as a whole. *Strong enough* is defined by euristics.

The main reason for having two phases is avoiding ants to pick always the same few edges just because they have been selected once or two, and being among the very few holding a probably small amount of pheromone. So basically befor start chosing a way, we want to be sure a reasonable number of ants have been walking and leaving feromone.

Observing ants in nature provides the intuition that they move in an almost random way at start, and they converge to the right way after a while, wehen feromone is strong and stable enough.

Ants are divided into two colonies, competing to find their best solution. The two colonies will provide the two solutions required by the Kaggle challenge. It is worth to note, as highlighted in the paper, that the first colony release a more powerful feromone in order to avoid real competition, and have one colony to converge faster over the "best" solution, and the second follow in the convergence process to the "second best" solution.

A distinctive characteristics of this solution is cluster management. Clusters are used to reduce the complexity of the problem (the number of nodes and edges) but, instead of defining cluster statically, we use ants to generate clusters. As soon a prime edge is identified, its source node (or source cluster) and its target node (or target cluster) are merged in a single cluster.

<h2>Data representation</h2>

As said, the main data structure for the algorithm is a graph, implemented as a *list of dictionaries in Python*. An element (a dictionary) has a *node name as the key* and an *inner dictionary as the value*, holding the edges structure for that node.

The inner dictionary has an *edge name as the key*, and a *second inner dictionary as the value*, holding the following leaf information:

1. key <\'lb\'> holds a label for the edge, that can be <way> (the edge is clear and can be selected by ants), <way1> (the edge has been assigned to the final solution for the first colony), <way2> (similar to <way1>) and <blocked> (when an edge is assigned to the final solution, the inverse edge is retired to cope with Kaggle's requirement).

2. key <\'nn\'> holds the target node for the edge.

3. key <\'w\'> holds the edge's weight.

4. key <\'f1\'> holds the feromone left by the first colony. 

5. key <\'f2\'> holds the pheromone left by the second colony. 

This release of the software is strongly sequential, as it runs ants from the two colonies one by one. Clearly the software could be improved by exploiting parallelism, by running ants concurrently. It is worth to note that the data structure for the edge can hold the feromone for two distinct colonies, and colony is a parameter in almost all (if not all) procedures. Although this is pretty useless in the current, deeply sequential, release of the software, it permits to exploit parallelism in a second wave. It would be necessary to run ants in parallel from a multi-threaded host (i.e. a few ants per thread), and modify the graph data structure to permit concurrency (i.e. by lockin resources during updates).

The following piece of code reads the source CSV file and generates a list of dictionaries with a key for nodes and empty values for edges. With 150.000 nodes, possible edges are 22,5 billions. Instead of generating all of them, ants are levereged to generate only the most useful ones during the exploration phase.

In [ ]:
N = {}
with open('c:\source.csv') as f:
    rows = csv.DictReader(f)
    for r in rows:
        node = {'id':r['id'], 'x':int(r['x']), 'y':int(r['y'])}
        N.append(node)

g = {}
tot_nodes = len(N)
for i in range(tot_nodes):
    client[N[i]['id']]={}

Before starting exploring ants we calculate an important parameter (**alpha**), that will be used to normalize the calculation of the amount of feromone to leave on a certain edge. This is important to avoid too small amounts that might be rounded to zero. 

The feromone to leave on a certain edge <e>, being part of a valid cheminy <c> is given by the following formula:

f = \frac{w(\theta)}{w(c)}*\frac{\theta}{n * w(e)}

where \w is a function that returns the weight, \theta is the baseline solution, \c is a cheminy, \e is an edge and \n is the number of nodes in the graph.

A baseline solution is calculated with the procedure <baseline_perf>. It is determined with a simple space filling function, assuming cities to be distributed uniformally in *n* (<n = 150000>) rectangles of size dx * dy.

After having calculated <alpha> the amount of feromone to leave on a certain edge is computed as <ff = alpha / (weight * all_weight)>, where <weight> is the weight of the edge, and <all_weight> is the weight of the full path.

In [ ]:
def baseline_perf(N): #calculates the weight of the baselline solution
    max_x = max(N, key=lambda r:r['x'])
    max_y = max(N, key=lambda r:r['y']) 
    dy = max_y['y'] / math.sqrt(len(N))
    dx = max_x['x'] / math.sqrt(len(N))
    return math.sqrt(dx**2 + dy**2) * len(N)
 
alpha = baseline_perf(N)**2 / len(N)

<h2>Exploring ants</h2>

The following piece of code fires ants during the exloration phase, where <e_ant> implement an exploring ant. It finds a complete cheminy from the origin node back to the origin node, by touching all nodes exactly once. 

<e_ants> gets the following parameters:
- <g> is a pointer to the graph
- <N> is the pointer to the list of nodes and their (x,y) coordinates
- <origin> is the name of the origin node
- <colony1> and <colony2> are constants indicating the name of the colony
- <alpha> is a constant used to calculate the pheromont amount to leave
It returns the total lenght of the solution identified. In addition <e_ant> leave feromone on edges that will be leveraged by  the hunting phase.

In [ ]:
for j in range(num_iterations):
        w1 = e_ant(g, N, origin, colony1, alpha)
        w2 = e_ant(g, N, origin, colony2, alpha)

Exploring ants are run a finite number of times (<num_iterations>). Although it has been noticed with a reduced data set that after a certain amount of iterations the average number of edges per node gets stable (and feromone then start to be accumulated on that set of edges mainly), for the full data set and the computing resources at our disposal we have not been able to identify that critical point. So we limited the number of iterations to a reasonable amount based on computing time (one single exploring ant takes circa 210 seconds).

The code of the exploring ant is implemented in the following iterative procedure:
- it first find a valid path, that starts from origin and come back to origin after having touched all the nodes exactly once
- while moving forward it calculates the total weight of the cheminy, by summing up the weight of each edge
- then the cheminy is visited back and feromone is left accordingly to the formula described above

It is worth to note that we leave feromone only after having identified a valid path.

The procedure employs the following data structures:
- <vis> is a list of booleans, indicating if a node has been visited or not during a certain iteration
- <stk> is a stack for storing the cheminy steps while moving forward
- <n> stores the number of nodes added to the cheminy, and it is used for the termination condition <n \< len(g)>
- <node> stores the current node under evaluation
- <all-weight> stores the accumulated weight of the cheminy
- <NN>, <x> and <y> stores the cities and their coordinates
- <T> stores a querable representation of the space


In [ ]:
def e_ant(g, N, origin, colony, alpha):
    
    vis = [False] * len(g) 
    stk = [] 

    node = origin
    all_weight = 0
    n = 1 

    NN = [{'id':N[i]['id'], 'x':N[i]['x'], 'y':N[i]['y']} for i in range(len(N))]
    x = [N[i]['x'] for i in range(len(N))]
    y = [N[i]['y'] for i in range(len(N))]
    T = cKDTree(np.c_[x, y])
    
    # walk forward through the graph and stack the steps of a valid path
    while n < len(g):  
        
        if (n % tree_refresh_interval) == 0: #refresh tree
            NN, x, y, T = refresh_tree(NN, x, y, vis)
        
        # get the successor for node, insert it into the stack, mark the node as visited in vis 
        # increase all_weight and increase counters
        n_edge = query_edges(T, N[int(node)]['x'], N[int(node)]['y'], NN, vis, node, origin, n)
        stk.append((node, n_edge))
        vis[int(node)] = True
        all_weight += n_edge['w']
        node = n_edge['nn']
        n += 1
    
    #sum up the last node weight
    n = int(node)
    o = int(origin)
    all_weight += math.sqrt((N[n]['x']-N[o]['x'])**2 + (N[n]['y']-N[o]['y'])**2)

    # start the trip back, popping the stack
    while stk != []:
        s = stk.pop() 
        s_node = s[0]
        edge = s[1]['edge']
        t_node = s[1]['nn']
        w = s[1]['w'] 
        
        # upserts edges into the graph, relising the right amount of feromone
        if edge not in list(g[s_node].keys()): 
            add_edge(g, s_node, t_node, edge, 'way', w, 0, 0, 0) 
        release_feromone(g, s_node, edge, colony, 'increase', w, all_weight, alpha)

    return all_weight

It is worth to note that we used the [scipy.spatial.cKDTree] (https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.cKDTree.html) class to query nodes by distance. Given a root node, the query method provides a list of nodes and their distance from the root node, sorted by distance.

>scipy.spatial.cKDTree "provides an index into a set of k-dimensional points which can be used to rapidly look up the nearest neighbors of any point. The algorithm used is described in Maneewongvatana and Mount 1999. The general idea is that the kd-tree is a binary trie, each of whose nodes represents an axis-aligned hyperrectangle. Each node specifies an axis and splits the set of points based on whether their coordinate along that axis is greater than or less than a particular value".

The criteria behind the <e_ant> algorithm is that it starts from the origin and for each node selects randomly the next node among the closest that have not already been visited. This has been implemeted in the <query_edges> procedure that returns the successor for a node with coordinates <(node_x, node_y)>:
- <NN> holds all the nodes that have not already been visited
- <T> is the tree

In addition the following parameters are used:

In [ ]:
tree_refresh_interval = 10000 #every how many iterations to refresh the tree
max_randomity = 10 #number of edges, among those leading to non visited nodes, to chose from in a random way
tree_batch_size = 100 #size of the list of successors to read within a single query to the tree
num_iterations = 10 #number of ants to explore the graph

In [ ]:
def query_edges(T, node_x, node_y, NN, vis, node, origin, n):
    i = 0
    nn_list = []
    while nn_list == [] or i == 0:
        i = i + 1 
        batch_nodes = range(1+tree_batch_size * (i-1), min(1+tree_batch_size * i, len(NN)+1))
        ww, ii = T.query([node_x, node_y], k=batch_nodes,n_jobs=-1) #i-th closest
        n_list = [NN[i]['id'] for i in ii]
        nn_list = list(filter(lambda x: not vis[int(x)] and x not in (origin, node), n_list))

    nn_idx = int(random.uniform(0,min(max_randomity, len(nn_list)-1)))
    nn = nn_list[nn_idx]
    e = 'e_' + node + ':' + nn        
    return {'edge':e, 'nn':nn, 'w':ww[nn_idx]}

<max_randomity> is largerly responsible for how many edges are generated per node, and it interacts with how many exploring ants are run (<num_iterations>). The larger they are, the larger is the average number of edges generated per node. With 100 ants and <max_randomity = 3> we had *circa 22 edges per node in average*. This is very important because the hunting ants will fail to identify a cheminy if they will rely on too few edges per node. Reasons for this stay behind the combinatorial nature of the TSP problem. It would be worth as a future work to study the distribution of discovered edges over iterations, and a model to size the exploring phase in order to have the hunting phase converging. For this implementation we simply let hunting ants to query again the tree when they run out of edges.

It is worth to note that every 10.000 nodes discovered the tree is refreshed for performance reasons: all the nodes already visited are cancelled from <NN>, <x> and <y>, and the tree recomputed. We obtained significant performance improvement, but we need to realign the indexes returned by the tree to the the graph data structure (that hold all the nodes vs. a subset of them).

In [ ]:
def refresh_tree(NN, x, y, vis):
    x = [x[i] for i in range(len(x)) if not vis[int(NN[i]['id'])]]
    y = [y[i] for i in range(len(y)) if not vis[int(NN[i]['id'])]]
    NN = [NN[i] for i in range(len(NN)) if not vis[int(NN[i]['id'])]] 
    T = cKDTree(np.c_[x, y])
    return NN, x, y, T

Feromone is released when the ant gets back, once a valid cheminy has been identified. The following fragment of code represents where this work is done within the <release_feromone> procedure. The <direction> parameter controls the scenario underneath feromone is released. <fer1> and <fer2> controls the power of feromone left by the two colonies.

In [ ]:
fer1 = 1 #feromon power for first colony 
fer2 = 0.5 #feromon power for second colony 

def release_feromone (graph, node, edge, colony, direction, weight, all_weight, alpha):
        ...
    #feromone left by exploring ants
    elif direction == "increase":
        ff = alpha / (weight * all_weight)
        if colony == colony1:
            update_edge(graph, node, edge, f1 = read_edge_prop(graph, node, edge, f) + (ff*fer1))
        else:
            update_edge(graph, node, edge, f2 = read_edge_prop(graph, node, edge, f) + (ff*fer2))


In [ ]:
<h2>Hunting ants</h2>

After the exploring ants finish running, it is the turn of the hunting ants to run. They will leverage the feromone left by exploring ants to identify the correct solutions. Hunting ants are launched still in a very sequential way from the following calls to <h_ant>, that returns *True* once all the edges included in a solution can be considered **strong enough**, hence marked <way1> for the first colony or <way2> for the second colony. The strongness of an edge is based on euristics and feromone.

In [ ]:
term = False
while not term:
        term, weight = h_ant(g, N, origin, colony1, alpha)

term = False
while not term:
        term, weight = h_ant(g, N, origin, colony2, alpha) 

<h_ant> is based on an iterative, posticipated navigation of the graph. The principal variables and parameters are the ones in the initialization part of the procedure. They are all similar to parameters and variables employed in <e_ant>.

In [ ]:
dim_batch = 6

def h_ant (g, N, origin, colony, alpha):
    
    stk = [] 
    clu = 'clu1' if colony == colony1 else 'clu2' 
    way = 'way1' if colony == colony1 else 'way2'
    
    node = origin
    n = 0
    all_weight = 0

    NN = [{'id':N[i]['id'], 'x':N[i]['x'], 'y':N[i]['y']} for i in range(len(N))]
    x = [N[i]['x'] for i in range(len(N))]
    y = [N[i]['y'] for i in range(len(N))]
    T = cKDTree(np.c_[x, y])

    forced = False    

The idea behind <h_ant> is the following:
    + during the **anticipated visit** of the graph it first identify a valid path, with a anticipated navigation of the graph. Edges are chosed based on feromone.
    + at each step it increment the total weight of the path (<all_weight>), the number of nodes added to the solution (<n>) and mark the node as visited in <vis>. 
    + the state of each step is recorded into the stack <stk>. The core of the state is the edge chosed at that step to move forward.
    + once all the nodes have been added to the solution, the **posticipated visit** of nodes starts by popping the <stk> and it is checked if the edge is strong enough. In that case the edge is marked as <way1> for <colony1> (or <way2> for <colony2>) and a *cluster* is eventually formed (or managed if already existed. 

Clusters are group of nodes connected by strong edges. The first node of the cluster holds cluster configuration (number of nodes and total weight of the cluster), a pointer to the last node in the cluster (that provides the list of edges for the cluster) and it is the only node in the cluster that can be reached by other nodes (so it is marked as <False> in <vis>, while all the others gets marked as <True> *forever*).

<h3>Anticipated visit of the graph: node visited for the first time</h3>

The anticipated visit of <h_ant> takes on until all the nodes have not been visited. In case a <node> is visited for the first time (<not vis[int(node)]>) <scan_edges> is called with <batch_num = 0>. <scan_edges> returns the following results:

<cluster_config> is a dictionary describing the cluster structure, holding the following keys:
+ <n> is the number of nodes in the cluster
+ <w> is the weight of the cluster
+ <en> is the final node name, holding the list of edges for the cluster
Cluster configuration is held in the graph with a special *key* in the dictionary (<clu1> for <colony1> and <clu2> for <colony2>).

<tot_feromone> is the total amount of feromone that has been left by ants leaving from <node>. This will be useful for the euristic that determine if an edge is strong enough

<edges> is a list of <batch_size> edges, sorted by pheromone. A list element is a dictionary with the following keys:
    <e> is the name of the edge, i.e. <e_5:8> is a direct edge from node 5 to node 8
    <nn> is the target node name
    <w> is the weight of the edge
    <score> is the score attribuited to the edge based on feromone

As the number of outgoing edges might be large with 150.000 nodes, they are read in batches of <batch_size> size. Edges are tried in order, and when a batch is finished the next batch must be red. When the target node of an edge has an empty list of edges (i.e. al its targets have been already visited), the next edge is tried. 

A decision have been made here in balancing how many edges to include in a batch, and how often to build the edges list. In fact, as batches are ordered by feromone, every time we ask for a batch it is necessary to scan all the node's edges. Empirically it's been noticed that for the largest part of an ant life (an iteration of the algorithm), edges are discovered within the very firsts. For this reason we have set <batch_size = 6>.

In [ ]:
while n < len(g):
        #Every tree_refresh_interval nodes discovered refresh the tree
        if (n % tree_refresh_interval) == 0:
            log(verbose, 'Refreshing the tree...')
            NN, x, y, T = refresh_tree(NN, x, y, vis)
    
        # first time we visit the node
        if not vis[int(node)]:
            vis[int(node)] = True
            batch_num = 0
            cluster_config, tot_feromone, edges = scan_edges(g, vis, node, origin, colony, batch_num, dim_batch, 0) 
            # edges = [{edge}]; edge = {'e':e, 'nn':nn, 'w':w, 'score':sc)
            n = n + cluster_config['n']
            all_weight += (cluster_config['w'])

Next we check a series of conditions to understand how to move forward. 

In case <edges != []> a list of edges exist that leads to a valid target node. The first edge is selected, the state pushed to the stack and node set to <edges[0]['nn']> for the next iteration. It is worth to comment on the <stk> structure, that's a list of dictionaries with the following keys:
- <cn>: node's name.
- <edges>: current batch of <batch_size> edges.
- <clu>: cluster configuration for the current node.
- <batch_num>: the sequential id of the batch of edges. Initially it is <0>, meaning the first.
- <i>: the edge id in the batch. Initially it is <0>, meaning the first.
- <t_f>: total feromone

When <edges == []> and <n &lt len(g)> there are still nodes to be discovered in tha path, but the current node has no edges to follow (i.e. because they all lead to already visited nodes, or because they might be reserved to the other colony). On this case we get back to the previous node, and select a different target.

<forced> is a special case. It's been noticed that having enough edges per node so that a valid path can be discovered is tricky. In the initial release of the software, when all the edges didn't lead to somewhere, we simply disallocated nodes from the stack and try a different way. In the final part of the graph traversal, when nodes start to be scarse, the algorithm would start to go back and forward almost forever. In balancing such a long run (provided it ever finished) and allocating an oversised set of edges per node, we took the decision to kepp them small but stop stepping back when the list of edges terminates. In such a case we set <force = True> and the following steps are *forced*:
1. the current node strongest edge (the first in the first batch) and its target node are forced to be in the path
2. as its target node has an empty list of edges, a new edge is queried from the tree (<query_edges>) and added to the graph (<add_edge>) with <feromone = 0> for both colonies.

The final <else> branch is the termination condition, happening when <n &eq len(g)>. In that case we sum up the closing edge (returning to the origin) weight  and we initialize variables for the posticipated visit of the graph.

In [ ]:
            if forced: #all edges have been already tried with no success. Force a successor.
                forced = False
                edge = query_edges(T, N[int(node)]['x'], N[int(node)]['y'], NN, vis, node, origin, n) #return {'edge':e, 'nn':nn, 'w':ww[nn_idx]}
                add_edge(g, node, edge['nn'], edge['edge'], "way", edge['w'], 0, 0, 0)
                edges = [edge]

            if edges != []: #a list of edges exist, let's take the first one!
                all_weight += edges[0]['w']
                stk.append({'cn':node, 'edges':edges, 'clu':cluster_config, 'batch_num':0, 'i':0, 't_f':tot_feromone})
                node = edges[0]['nn']
            elif n < len(g): #a list of edges doesn't exists but there are still nodes to be discovered
                vis[int(node)] = False
                n -= cluster_config['n']
                all_weight -= (cluster_config['w'])
                s = stk.pop()
                node = s['cn']
            else: #and not force
                all_weight += math.sqrt((N[int(node)]['x']-N[int(origin)]['x'])**2 + (N[int(node)]['y']-N[int(origin)]['y'])**2)
                cluster_config = {'en':node, 'w':0, 'n':1} if clu not in g[node].keys() else g[node][clu]
                one_cluster = True
                vis[int(node)] = False 

<h3>Anticipated visit of the graph: node already visited</h3>

In case the current node is already visited (<vis[int(node)]>) it means that we already tried some of its edges, but they led to a blind way (target nodes with an empty list of edges to follow). What we need to do is selecting, if possible, a new edge or force a path.

We first pop the node's status from the stack, decrease <all_weight> of the weight of the last edge tried and failed, and pass to the next edge in the list (<i = i+1>). In case it was the last edge in the list (<i &gr; len(edges)-1>) we need to read the next batch from the graph by calling <scan_edges> again. 

In case <scan_edges> returns is an empty list (<edges &eq;&eq; []>) it means there are no more edges to try, and we are in the situation an edge have to be forced. So <scan_edges> might be called again to load the first edge in the first batch: that's the best edge we have in terms of feromone, and that's the one we want to force in the path. Finally we set <forced &eq; True> so that the next iteration happening on the target of the forced edge, will know that although that node has an empty list of edges, it must be forced to the path by pulling a new edge from the tree (see code above).

This hibrid mechanism of prioritizing edges by feromone, and eventually pulling never seen edges from the tree in case of need, greately improved performance. A single ant run takes around 45 seconds on a laptop.

In [ ]:
        else:
            #s holds the stack element popped before
            cluster_config = s['clu']
            edges = s['edges']
            batch_num = s['batch_num']
            i = s['i']
            tot_feromone = s['t_f']

            all_weight = all_weight - edges[i]['w']           
            i = i+1 
            if i > len(edges)-1:
                batch_num += 1
                i = 0
                cluster_config, tot_feromone, edges = scan_edges(g, vis, node, origin, colony, batch_num, dim_batch, tot_feromone)
                    
            if edges == []:
                    #force the best edge 
                    if batch_num == 0: #first batch already loaded, just point to the first element                      
                        i = 0
                    else:
                        batch_num = 0
                        i = 0
                        cluster_config, tot_feromone, edges = scan_edges(g, vis, node, origin, colony, batch_num, dim_batch, tot_feromone)
                    
                    forced = True             

Whatever is the way we pick the next edge, we put it into the stack and iterate

In [ ]:
            all_weight += edges[i]['w'] 
            stk.append({'cn':node, 'edges':edges, 'clu': cluster_config,'batch_num':batch_num, 'i':i, 't_f':tot_feromone})
            node = edges[i]['nn'] 

In [ ]:
<h3>Posticipated visit of the graph</h3>

Once the anticipated visit of the graph has touched all the nodes in the graph, we get back by pulling edges from the stack and do some work:
- feromone is left
- we need to check if edges are strong enough
- for strong edges, clusters are managed
- we need to generate the termination condition for <h_ant> (being all edges in the path being strong)

In [ ]:
while stk != []:
        s = stk.pop()  
        #stk elemnt is {'cn':n, 'edges':es, 'clu':clu, 'batch_num':bn, 'i':i, 't_f':tf}
        #edges = [{'edge':e, 'nn':nn, 'w':w, 'score':score}]
        node = s['cn']
        node_config = s['clu']
        edge = s['edges'][s['i']]['edge']
        nn = read_edge_prop(g, node_config['en'], edge, 'nn')          
        
        tot_feromone = s['t_f']
        
        #release feromone
        release_feromone (g, node_config['en'], edge, colony, 'reward', read_edge_prop(g, node_config['en'], edge, 'w'), all_weight, alpha)
        
        #clustering
        vis[int(node)] = False 
        edg = read_edge(g, node_config['en'], edge)
        f = edg['f1'] if colony == colony1 else edg['f2']
        vs_f = edg['f2'] if colony == colony1 else edg['f1']

First the stack is popped, and we get a dictionary like <{'cn':n, 'edges':es, 'clu':clu, 'batch_num':bn, 'i':i, 't_f':tf}>. Feromone is left by calling the **reward** schema in <release_feromone>. The following piece of code is extracted form <release_feromone>: we basically reward ad edge in the path by increasing its feromone by <fer_reward>. This is one of the most critical parameters in the algorithms:
- with large values (say 10) a lower number of ants would be necessary to converge, because the algortims terminate only when al the edges in the path are strong, meaning an edge holds a certain quota of the total feromone for the node, and has a certain advantage over the competing colony
- with small values (say 0.5) we would get better results at the price of running for longer

In [ ]:
    if direction == "reward":
        if colony == colony1:
            ff = (alpha * fer1) / (weight * all_weight)  if read_edge_prop (graph, node, edge, 'f1') == 0 else read_edge_prop (graph, node, edge, 'f1')
            update_edge(graph, node, edge, f1=ff * fer_reward)
        else:
            ff = (alpha * fer2) / (weight * all_weight) if read_edge_prop (graph, node, edge, 'f2') == 0 else read_edge_prop (graph, node, edge, 'f2')
            update_edge(graph, node, edge, f2 = ff * fer_reward)

Moving forward on the main procedure we test if the edge is **strong enough** by using the following euristic:
    - it must be 3x stronger than the competing colony
    - it must account for at least 40\% of the total feromone of the node
    
In case the edge is strong it means that the current node and its target node belong to a cluster and we perform the following operations:
- update the cluster configuration <cluster_config =  {'en':en, 'w': w, 'n': n}>. It is worth to note that a new cluster configuration is initialized for the current node every time a not strong edge (<is_prime> is <False>) is encountered.
- delete cluster keys from the current and target nodes, as a new cluster key will be generated on the first node of the cluster once it will be encountered.
- assign the edge to the final solution, by marking it as <way1> (or <way2>). The other edges feromone will be reset to <0> in order to favour <colony2> to converge.
- block the inverse edge, as it can't be selected any more by either <colony1> or <colony2>. This is a Kaggle's requirement.

It is worth to note that when we start stepping back on the stack, we release every node (<vis[node]=False>) for the next iteration. In case a node is the source or the target of a strong edge, it is set again to <True>, unless it is the first node of a lcuster: this way the next iterations will only discovers clusters, not single nodes being part of a cluster, considerably speeding up processing.

In [ ]:
        is_prime = (f > 3*vs_f and f > 0.4*tot_feromone)

        if is_prime:
            en = cluster_config['en']
            w = node_config['w'] + cluster_config['w'] + edg['w']
            n = node_config['n'] + cluster_config['n']
            
            #if clusters already exist in the graph for either the current node or target node, delete them
            #a new cluster key will be created once we encounter the cluster's first node
            if clu in g[node].keys():
                del g[node][clu]
            
            if clu in g[nn].keys():
                del g[nn][clu]

            #update cluster_config and retire the target node from being pulled by next iterations
            cluster_config =  {'en':en, 'w': w, 'n': n}
            vis[int(nn)] = True
            
            #mark the edge as "way1" or "way2" and reset the feromone for the other edges in order to favour the competing colony
            update_edge(g, node_config['en'], edge, lb=way)
            release_feromone(g, node_config['en'], edge, colony, 'reset', 0, 0, 0)
            
            #mark the inverse edge as "blocked" - it can't be chosen any more
            for e in get_edges(g, nn):
                if read_edge_prop(g, nn, e, 'nn') == node_config['en']:
                    update_edge(g, nn, e, lb="blocked")
                    break  

In case the edge is not strong, it means that it represents a border between two different clusters (eventually of one node!). So the current node target is the first node of a cluster, and we will generate a graph key with the cluster configuration for the target node (how many nodes it encompass, what's the total weight of its edges and who is the final node holding the list of successors).

In [ ]:
        else:
            if cluster_config['en'] != nn:
                g[nn][clu] = cluster_config 
            cluster_config = node_config

Finally we check the termination condition and the procedure returns. The termination condition fires when all the edges in the path are strong. It is worth to note that when this condition happens, we have one single cluster holding all the nodes and the selected edges. Those edges are recognizable easily because they are marked as <way1> or <way2> on the graph.

In [ ]:
        #termination condition
        one_cluster = one_cluster if is_prime else False

    return one_cluster, all_weight

<h3>Scan_edges</h3>

An important procedure that has been used by <h-ant> is <scan_edges>, that for a certain node selects a batch of edges sorted by feromone and returns the total feromone for the edge and the node's cluster configuration.

As a first thing <scan_edges> checks if <node> is part of a cluster by checking if a <clu1> key for <colony1> or a <clu2> key for <colony2> exists. In case it does it pull from the graph the node's cluster configuration. If it doesn't it means that the node doesn't belong to any cluster, so its cluster configuration is initialized.

In [ ]:
def scan_edges(graph, vis, node, origin, colony, batch_num, dim_batch, tot_feromone):
    if 'clu1' in list(graph[node].keys()) and colony == colony1:
        node_edges = graph[node]['clu1']['en']
        clu_w = graph[node]['clu1']['w']
        clu_n = graph[node]['clu1']['n']
    elif 'clu2' in list(graph[node].keys()) and colony == colony2:
        node_edges = graph[node]['clu2']['en']
        clu_w = graph[node]['clu2']['w']
        clu_n = graph[node]['clu2']['n']
    else:
        node_edges = node
        clu_w = 0 
        clu_n = 1
    
    clu = {'en':node_edges, 'n':clu_n, 'w':clu_w}

If <tot_feromone> parameter is zero, then total feromone should be calculated. It basically requires to scan all the outgoing edges of the node, and sum up feromone for the two colonies for all viable edges (basically those not marked as either <blocked> or <way2> for <colony1> (or <way1> for <colony2>)

In [ ]:
    if tot_feromone == 0:
        tot_f1, tot_f2 = 0, 0
        for e in get_edges(graph, node_edges):
            edg = read_edge(graph, node_edges, e)
            lb = edg['lb'] 
            nn = edg['nn'] 
            
            if nn != origin and ((colony == colony1 and lb not in ('blocked', "way2")) or (colony == colony2 and lb not in ('blocked', 'way1'))):
                    tot_f1 += edg['f1'] 
                    tot_f2 += edg['f2'] 
    else:
        tot_f1, tot_f2 = tot_feromone, tot_feromone

Next we need to collect the list of edges for the node. It is basically a simple cycle over all the node's edges in the graph where we only pick the viable ones (basically those **not already visited** and not marked as either <blocked> or <way2> for <colony1> (or <way1> for <colony2>)

In [ ]:
    edges = []
    edge = {}
    f1, f2 = 0, 0
    tot_f = tot_f1 if colony == colony1 else tot_f2

    for e in get_edges(graph, node_edges):  
        edg = read_edge(graph, node_edges, e)
        lb = edg['lb'] 
        nn = edg['nn']
        if not vis[int(nn)] and lb != 'blocked' and nn != origin and ((colony == colony1 and lb != "way2") or (colony == colony2 and lb != "way1")):
            #nn = edg['nn'] 
            w = edg['w'] 
            f1 = edg['f1'] 
            f2 = edg['f2'] 
            score = 0
            
            

For each viable edge a score is calculated by employng the following euristic:
- if the edge is strong it gets a random score in <(16, 20)>
- if the edge is not strong enough vs. the competing colony, but account for at least 40\% of the total feromone in the node, then it get a random score in <(11, 15)>
- if the edge is strong enough vs. the competing colony, but doesn't account for at least 40\% of the total feromone in the node, then it get a random score in <(6, 10)>
- if the edge's feromone is just at least 80\% of the competing colony it is scored in <(1, 5)>

Basically we wanted to prioritize all the strong edges first, then those that are strong in their colony (but not vs. the other one), then those that are stronger enough than the competition but with an amount of uncertainty regarding the colony winner, and then all the rest.

in case there was more edges belonging to the same profile, the decision is random.

In [ ]:
            f = f1 if colony == colony1 else f2
            vs_f = f2 if colony == colony1 else f1  
                    
            if f > 3*vs_f and f > 0.4*tot_f:
                score = random.uniform(16,20)
            elif (f <= 3*vs_f and f > 0.4 * tot_f):
                score = random.uniform(11,15)
            elif (f > 3*vs_f and f <= 0.4*tot_f):
                score = random.uniform(6,10)    
            elif f > 0.8*vs_f:
                score = random.uniform(1,5)                    
            else:
                score = 0
                
            edge = {'edge':e, 'nn':nn, 'w':w, 'score':score}
            edges.append(edge) 

    

Finally we return the right batch of edges after having sorted the list by score or, eventually, an empty list.

In [ ]:
if batch_num * dim_batch > len(edges): #left end of the required batch follow over the right end of the list of edges 
        return clu, tot_f, []
    else:
        edges.sort(key = lambda t: t['score'], reverse = True)
        return clu, tot_f, edges[batch_num*dim_batch:(batch_num+1)*dim_batch]